In [4]:
! pip3 install --upgrade --quiet anthropic # i believe this is not needed
! pip3 install --upgrade --quiet 'anthropic[vertex]'

# restart the kernel after this step

In [1]:
from anthropic import AnthropicVertex

In [6]:
# params
P = ! gcloud config list --format 'value(core.project)'
PROJECT_ID = P[0]

In [5]:
# client
client = AnthropicVertex(project_id=PROJECT_ID, region="us-east5")

# Simple example with two tools to force single pass calling
message = client.messages.create(
    model="claude-sonnet-4",
    max_tokens=1024,
    tools=[
        {
            "name": "search_restaurants",
            "description": "Search for restaurants in a location",
            "input_schema": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City or area to search",
                    },
                    "cuisine": {
                        "type": "string", 
                        "description": "Type of cuisine"
                    }
                },
                "required": ["location"],
            },
        },
        {
            "name": "get_weather",
            "description": "Get current weather for a location",
            "input_schema": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City name",
                    }
                },
                "required": ["location"],
            },
        }
    ],
    messages=[
        {
            "role": "user",
            "content": "I'm visiting San Francisco today. Find me Italian restaurants and check the weather so I know if I should eat outdoors.",
        }
    ],
)

# Check if multiple tools were called in single pass
tool_calls = [block for block in message.content if hasattr(block, 'type') and block.type == "tool_use"]
print(f"Number of tools called: {len(tool_calls)}")
print(f"Single pass calling: {'Yes' if len(tool_calls) > 1 else 'No'}")

print("\nFull response:")
print(message.model_dump_json(indent=2))

Number of tools called: 2
Single pass calling: Yes

Full response:
{
  "id": "msg_vrtx_01B7auvi369cxuKSa46Y69ma",
  "content": [
    {
      "citations": null,
      "text": "I'll help you find Italian restaurants in San Francisco and check the current weather so you can decide about outdoor dining.",
      "type": "text"
    },
    {
      "id": "toolu_vrtx_01VjERLLeVM1vhe9RdcoiPmq",
      "input": {
        "location": "San Francisco",
        "cuisine": "Italian"
      },
      "name": "search_restaurants",
      "type": "tool_use"
    },
    {
      "id": "toolu_vrtx_01DvtjUfuX33Hgi5Um1zobXw",
      "input": {
        "location": "San Francisco"
      },
      "name": "get_weather",
      "type": "tool_use"
    }
  ],
  "model": "claude-sonnet-4-20250514",
  "role": "assistant",
  "stop_reason": "tool_use",
  "stop_sequence": null,
  "type": "message",
  "usage": {
    "cache_creation_input_tokens": 0,
    "cache_read_input_tokens": 0,
    "input_tokens": 488,
    "output_tokens": 